In [2]:
import os
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
MODEL = 'llama3.2'

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
website = Website("https://staging.ianwhinkle.com/")
get_links_user_prompt(website)

'Here is the list of links on the website of https://staging.ianwhinkle.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\nLinks (some might be relative links):\nhttps://ianwhinkle.com\nhttps://ianwhinkle.com/\nhttps://ianwhinkle.com/profile/\nhttps://ianwhinkle.com/experience/\nhttps://ianwhinkle.com/contact/\nhttps://github.com/NarwhalPrince/\nhttps://linkedin.com/in/ianwhinkle\n/profile\n/experience\n/contact\nhttps://ianwhinkle.com\n#top'

In [11]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [16]:
get_links("https://staging.ianwhinkle.com/")

{'links': [{'type': 'about page',
   'url': 'https://staging.ianwhinkle.com/about'},
  {'type': 'company page', 'url': 'https://staging.ianwhinkle.com/'},
  {'type': 'contact page', 'url': 'https://staging.ianwhinkle.com/contact/'}]}

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
user_prompt = get_brochure_user_prompt("Ian Hinkle", "https://staging.ianwhinkle.com/")

# it crashed with get_brochure_user_prompt being called in stream_brochure, but works if I get the prompt ahead of time
# obviously the function is now ignoring the parameters..

Found links: {'links': [{'type': 'home page', 'url': 'https://staging.ianwhinkle.com'}, {'type': 'company home page or About page', 'url': 'https://staging.ianwhinkle.com'}, {'type': '_experience'}]}


KeyError: 'url'

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Ian Hinkle", "https://staging.ianwhinkle.com/")

# Ian Hinkle: Bringing Excellence to Every Project

## About Us

Ian Hinkle is a seasoned Security Analyst with a passion for resolving complex issues. With a strong background in security and detail-oriented approach, Ian brings unparalleled expertise to every project.

### Company Overview

We are MadCap Software, Inc., a Texas State University-affiliated organization dedicated to delivering top-notch services. Our commitment to excellence drives us to create innovative solutions that meet the unique needs of our clients.

## What We Do

As an industry expert in security, Ian has extensive experience supporting various projects. His skills and expertise cover:

*   Security analysis
*   Complex issue resolution
*   Project management

### Our Mission

At MadCap Software, Inc., we strive to create exceptional experiences for our clients and team members alike. We aim to foster a culture of innovation, collaboration, and inclusivity.

## Join Our Team

If you share our passion for delivering top-notch services, we encourage you to explore job opportunities with us.

### Contact Us

Ready to connect? Reach out to us at:

 Phone: +1 737-342-6788
 Email: [email protected](mailto/email@IAN-HINCKLE.com)

Let's work together to create something amazing!

| --- | --- |
| **LinkedIn Profile** | [View Profile](https://www.linkedin.com/in/ianhinkle/) |
| **Personal Website** | [Visit Site](https://ian-hingrouple.com) |

## Get in Touch

We can't wait to collaborate with you!